In [45]:
import pandas as pd

In [46]:
import os

In [47]:
from decode_sensor_binary_log import PingViewerLogReader

In [48]:
from datetime import datetime, timedelta, time

In [49]:
from pathlib import Path

In [50]:
filename = "20240213-131400708"

In [51]:
os.chdir('c:\\Users\\leasa\\Dropbox\\wham')

In [52]:
import csv

# TODO: put in the path to the log file you want to process
#  e.g. "~/My Documents/PingViewer/Sensor_Log/something.bin"

In [53]:
logfile = Path(f"wham_data/{campaign}/navigation/ping_logs/{filename}.bin") 

In [54]:
from datetime import datetime, timedelta, time

In [55]:
def to_timedelta(time_str: str) -> timedelta:
    ''' Returns a time delta from an iso-format time string. '''
    delta = time.fromisoformat(time_str)
    return timedelta(hours=delta.hour, minutes=delta.minute,
                     seconds=delta.second, microseconds=delta.microsecond)

In [56]:
log = PingViewerLogReader(logfile)

In [57]:
outfile = Path(f"wham_data/{campaign}/navigation/ping_logs/{filename}").with_suffix(".csv")

In [58]:
# ideally it would be good to localise this to your timezone,
#  but in this case it shouldn't cause issues unless you were
#  recording the sonar data at the time of a daylight savings
#  changeover or something
start_time = datetime.strptime(logfile.stem, "%Y%m%d-%H%M%S%f")

In [59]:
with outfile.open("w") as out:
    csv_writer = csv.writer(out)
    csv_writer.writerow(("timestamp", "distance [mm]", "confidence [%]"))
    for timestamp, message in log.parser():
        # convert the 'duration since scanning started' into a local-time timestamp
        #  the .replace here ensures cross-compatibility between operating systems
        timestamp = start_time + to_timedelta(timestamp.replace('\x00',''))
        csv_writer.writerow((timestamp, message.distance, message.confidence))